In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [3]:
files = [f'{folder_path}test_identity.csv', 
         f'{folder_path}test_transaction.csv',
         f'{folder_path}train_identity.csv',
         f'{folder_path}train_transaction.csv',
         f'{folder_path}sample_submission.csv']

In [4]:
%%time
def load_data(file):
    return pd.read_csv(file)

with multiprocessing.Pool() as pool:
    test_id, test_tr, train_id, train_tr, sub = pool.map(load_data, files)

CPU times: user 1.62 s, sys: 2.48 s, total: 4.1 s
Wall time: 18.6 s


In [5]:
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
test = pd.merge(test_tr, test_id, on='TransactionID', how='left')

del test_id, test_tr, train_id, train_tr
gc.collect()

11

In [6]:
useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                   'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                   'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

In [7]:
cols_to_drop = [col for col in train.columns if col not in useful_features]
cols_to_drop.remove('isFraud')
cols_to_drop.remove('TransactionID')
cols_to_drop.remove('TransactionDT')

In [8]:
print('{} features are going to be dropped for being useless'.format(len(cols_to_drop)))

train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

152 features are going to be dropped for being useless


In [9]:
# New feature - decimal part of the transaction amount
train['TransactionAmt_decimal'] = ((train['TransactionAmt'] - train['TransactionAmt'].astype(int)) * 1000).astype(int)
test['TransactionAmt_decimal'] = ((test['TransactionAmt'] - test['TransactionAmt'].astype(int)) * 1000).astype(int)

# Count encoding for card1 feature. 
# Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
train['card1_count_full'] = train['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))
test['card1_count_full'] = test['card1'].map(pd.concat([train['card1'], test['card1']], ignore_index=True).value_counts(dropna=False))

# https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)
train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24
test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    train[feature] = train[f1].astype(str) + '_' + train[f2].astype(str)
    test[feature] = test[f1].astype(str) + '_' + test[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train[feature].astype(str).values) + list(test[feature].astype(str).values))
    train[feature] = le.transform(list(train[feature].astype(str).values))
    test[feature] = le.transform(list(test[feature].astype(str).values))
    
for feature in ['id_34', 'id_36']:
    if feature in useful_features:
        # Count encoded for both train and test
        train[feature + '_count_full'] = train[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
        test[feature + '_count_full'] = test[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
        
for feature in ['id_01', 'id_31', 'id_33', 'id_35', 'id_36']:
    if feature in useful_features:
        # Count encoded separately for train and test
        train[feature + '_count_dist'] = train[feature].map(train[feature].value_counts(dropna=False))
        test[feature + '_count_dist'] = test[feature].map(test[feature].value_counts(dropna=False))

In [10]:
for col in tqdm_notebook(train.columns):
    if train[col].dtype == 'object':
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))

In [11]:
print(train.shape, test.shape)

(590540, 301) (506691, 300)


In [12]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from kaggleKID.preprocessing import check_dataframe

df_train, df_test, diff_type_cols, not_numbic_cols, na_existed_cols, all_na_cols, trn_tst_imbalance_col = check_dataframe(train, test,['TransactionID','TransactionDT'])

card2 8933 8654 float64 [100.0, 100.0] [600.0, 600.0]
card3 1565 3002 float64 [100.0, 100.0] [231.0, 232.0]
card5 4259 4547 float64 [100.0, 100.0] [237.0, 237.0]
addr1 65706 65609 float64 [100.0, 100.0] [540.0, 540.0]
addr2 65706 65609 float64 [10.0, 10.0] [102.0, 102.0]
dist1 352271 291217 float64 [0.0, 0.0] [10286.0, 8081.0]
C1 0 3 float64 [0.0, 0.0] [4685.0, 2950.0]
C2 0 3 float64 [0.0, 0.0] [5691.0, 3275.0]
C4 0 3 float64 [0.0, 0.0] [2253.0, 1601.0]
C5 0 3 float64 [0.0, 0.0] [349.0, 376.0]
C6 0 3 float64 [0.0, 0.0] [2253.0, 1601.0]
C7 0 3 float64 [0.0, 0.0] [2255.0, 1621.0]
C8 0 3 float64 [0.0, 0.0] [3331.0, 1005.0]
C9 0 3 float64 [0.0, 0.0] [210.0, 572.0]
C10 0 3 float64 [0.0, 0.0] [3257.0, 881.0]
C11 0 3 float64 [0.0, 0.0] [3188.0, 2234.0]
C12 0 3 float64 [0.0, 0.0] [3188.0, 2234.0]
C13 0 4748 float64 [0.0, 0.0] [2918.0, 1562.0]
C14 0 3 float64 [0.0, 0.0] [1429.0, 797.0]
D1 1269 6031 float64 [0.0, 0.0] [640.0, 641.0]
D2 280797 234769 float64 [0.0, 0.0] [640.0, 641.0]
D3 262878 20

V234 449124 369375 float64 [0.0, 0.0] [121.0, 322.0]
V238 449124 369375 float64 [0.0, 0.0] [23.0, 12.0]
V239 449124 369375 float64 [0.0, 0.0] [23.0, 12.0]
V242 460110 379963 float64 [0.0, 0.0] [20.0, 27.0]
V243 460110 379963 float64 [0.0, 0.0] [57.0, 54.0]
V244 460110 379963 float64 [0.0, 0.0] [22.0, 33.0]
V245 449124 369375 float64 [0.0, 0.0] [262.0, 124.0]
V246 460110 379963 float64 [0.0, 0.0] [45.0, 224.0]
V247 460110 379963 float64 [0.0, 0.0] [18.0, 11.0]
V249 460110 379963 float64 [0.0, 0.0] [22.0, 19.0]
V251 449124 369375 float64 [0.0, 0.0] [18.0, 12.0]
V253 460110 379963 float64 [0.0, 0.0] [163.0, 96.0]
V256 449124 369375 float64 [0.0, 0.0] [87.0, 53.0]
V257 460110 379963 float64 [0.0, 0.0] [48.0, 224.0]
V258 460110 379963 float64 [0.0, 0.0] [66.0, 269.0]
V259 449124 369375 float64 [0.0, 0.0] [285.0, 131.0]
V261 460110 379963 float64 [0.0, 0.0] [49.0, 89.0]
V262 460110 379963 float64 [0.0, 0.0] [20.0, 34.0]
V263 460110 379963 float64 [0.0, 0.0] [153600.0, 1065496.5]
V264 460110 

KeyboardInterrupt: 

In [ ]:
print(df_train.shape, df_test.shape)
df_train.columns.tolist()

In [ ]:
for col in [col for col in diff_type_cols if col in df_test.columns]:
    df_train[col] = df_train[col].astype(df_test[col].dtype)
    print(col, df_train[col].dtype)

In [ ]:
df_train.to_pickle(f'{folder_path}lgb-single-model-lb-0-9419_train.pkl', compression='gzip')
df_test.to_pickle(f'{folder_path}lgb-single-model-lb-0-9419_test.pkl', compression='gzip')

In [ ]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']
test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)

In [ ]:
X.shape, test.shape

In [ ]:
# train.to_pickle(f'{folder_path}lgb-single-model-lb-0-9419_train.pkl', compression='gzip')
# test.to_pickle(f'{folder_path}lgb-single-model-lb-0-9419_test.pkl', compression='gzip')

In [ ]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [14]:
folds = TimeSeriesSplit(n_splits=5)

aucs = list()
feature_importances = pd.DataFrame()
feature_importances['feature'] = X.columns

training_start_time = time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
    start_time = time()
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
    
    feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
    aucs.append(clf.best_score['valid_1']['auc'])
    
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
print('-' * 30)
print('Training has finished.')
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
print('Mean AUC:', np.mean(aucs))
print('-' * 30)

Training on fold 1
Training until validation scores don't improve for 500 rounds.


KeyboardInterrupt: 

In [ ]:
feature_importances['average'] = feature_importances[['fold_{}'.format(fold + 1) for fold in range(folds.n_splits)]].mean(axis=1)
feature_importances.to_csv('feature_importances.csv')

plt.figure(figsize=(16, 16))
sns.barplot(data=feature_importances.sort_values(by='average', ascending=False).head(50), x='average', y='feature');
plt.title('50 TOP feature importance over {} folds average'.format(folds.n_splits));